# Exemple de manipulation de la librairie pyodk

Ressources :
https://github.com/getodk/pyodk
https://github.com/getodk/pyodk/tree/master/examples

Dans cet exemple nous illustrons les cas suivants :
 * Récupération des données des soumissions 
 * Récupération des attachments des soumissions
 * Changement de la variable reviewState des soumissions récupérés
 * Mise à jour des attachments d'un formulaire


## Utilisation de l'api odk central

Installer le paquet pyodk
`pip install pyodk`

Créer un fichier config.toml qui permet d'établir la connexion avec ODK central

```
[central]
base_url = "https://www.example.com"
username = "my_user"
password = "my_password"
default_project_id = 123
```

In [ ]:
from pyodk.client import Client
# Creation client odk central
client = Client(config_path="./config.toml")

: 

In [ ]:
# CONSTANTES
FORM_ID="mon_form"
PROJECT_ID=1

# Récupération des données du formulaire

https://odkcentral.docs.apiary.io/#reference/odata-endpoints/odata-form-service/data-document

In [ ]:

#   expand="*" => avec toutes les niveaux hiérarchiques (tables repeat)
#   filter => Filtre sur les données dans l'exemple à partir d'une date ou du statut de la soumission
form_data = client.submissions.get_table(
    form_id=FORM_ID,
    project_id=PROJECT_ID,
    expand="*",
    #filter="__system/submissionDate ge 2022-12-06T14:56:00.000Z"
    filter= "__system/reviewState ne 'rejected'"
  )

print("Nombre de données de la requête", len(form_data['value']))

: 

# Mise à jour des status des données du formulaire

https://odkcentral.docs.apiary.io/#reference/submissions/submissions/updating-submission-metadata
 
> **Warning**
> !!!!  Nous n'avons pas réussi à utiliser le client pyodk pour cette action d'où l'utilisation de requests avec la récupération des paramètres via le client pyodk

In [ ]:
import requests
import json

# Liste des statuts
review_states = ["approved", "hasIssues", "rejected"]

# Récupération du token
token = client.auth.get_token(
            username=client.config.central.username,
            password=client.config.central.password,
        )

for data in form_data["value"]:
    # Pour chaque données récupérée dans la requête précédante mise à jour du reviewState
    review_submission_response = requests.patch(
        f"{client.config.central.base_url}/v1/projects/{PROJECT_ID}/forms/{FORM_ID}/submissions/{data['__id']}",
        data=json.dumps({"reviewState": review_states[2]}),
        headers={
            "Content-Type": "application/json",
            "Authorization": "Bearer " + token,
        },
    )
    print(review_submission_response.json())

: 

# Récupération des médias (attachments) d'une soumission



In [ ]:
# Upload form attachments
# https://odkcentral.docs.apiary.io/#reference/submissions/attachments/listing-expected-submission-attachments
# https://odkcentral.docs.apiary.io/#reference/submissions/attachments/downloading-an-attachment

for data in form_data["value"]:
    # Récupération de la liste des attachments d'une soumission
    attachments_list = client.get(f"projects/1/forms/Sicen/submissions/{data['__id']}/attachments")
    print("Nombre de médias" , {data['__id']}, len(attachments_list.json()))
    for att in attachments_list.json():
      # Téléchargement des médias
      img = client.get(f"projects/1/forms/Sicen/submissions/{data['__id']}/attachments/{att['name']}")
      with open(att['name'], 'wb') as out_file:
          out_file.write(img.content)


: 

# Mise à jour des médias attachés à un formulaire

La mise à jour ce fait en trois temps :
 * passage en draft du formulaire
 * post des fichiers médias
 * publication d'une nouvelle version du formulaire


> **Warning**
> !!! Pour le post des csv contournement => passage du contenu du fichier dans data nous n'avons pas réussi à passer directement le fichier en binaire


In [ ]:

from datetime import datetime

# Draft form
request = client.post(f"projects/{PROJECT_ID}/forms/{FORM_ID}/draft")


#  liste des attachments du formulaire
response = client.get(f"projects/{PROJECT_ID}/forms/{FORM_ID}/attachments")
print(response.json())

# Upload attachement
FILE_NAME="etudes.csv"
file_content = open(f"./form_attachments/{FILE_NAME}", "rb").read()
response = client.post(
  f"projects/{PROJECT_ID}/forms/{FORM_ID}/draft/attachments/{FILE_NAME}",
  data = file_content,
)

# Publish form
version_number=datetime.now()
response = client.post(f"projects/{PROJECT_ID}/forms/{FORM_ID}/draft/publish?version={version_number}")
print(response.json())

: 